In [6]:
########################################
# Base Inputs
########################################
import warnings
import pandas as pd
from datetime import datetime
pd.set_option('display.max_rows',2000)

In [7]:
########################################
# Importing Parking Tickets - TSV Method
########################################
'''
TSV can be downloaded from:
  https://data.lacity.org/A-Well-Run-City/Parking-Citations/wjz9-h9np
  
This code needs the raw data to be named 'Parking_Citations.tsv'
and to be in the same folder as the jupyter notebook
'''

df_raw = pd.read_csv('Parking_Citations.tsv',sep='\t',header=0)

df_raw.head()

,Ticket number,Issue Date,Issue time,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,Route,Agency,Violation code,Violation Description,Fine amount,Latitude,Longitude
0,4272349605,12/30/2015,2201.0,NaN,NaN,CA,201605.0,NaN,OLDS,PA,GN,3069 SAN MARINO ST,00403,54.0,80.56E4+,RED ZONE,93.0,6471840.7,1842349.7
1,4272349616,12/30/2015,2205.0,NaN,NaN,CA,201508.0,NaN,HOND,PA,WT,2936 8TH ST W,00403,54.0,80.56E1,WHITE ZONE,58.0,6473823.2,1843512.0
2,4272821512,12/30/2015,1725.0,NaN,NaN,CA,10.0,NaN,TOYT,PA,SL,301 LAUREL AV N,00401,54.0,5204A-,DISPLAY OF TABS,25.0,6451207.5,1850273.2
3,4272821523,12/30/2015,1738.0,WF74,NaN,CA,2.0,NaN,RROV,PA,BK,8321 3RD ST W,00401,54.0,88.13B+,METER EXP.,63.0,6449387.2,1849063.5
4,4272821534,12/30/2015,1807.0,13,NaN,CA,1.0,NaN,FORD,PA,GN,121 CROFT AVE,00401,54.0,80.58L,PREFERENTIAL PARKING,68.0,6448347.2,1849662.2


In [8]:
########################################
# Importing Agency Codes
########################################
'''
TSV can be downloaded from:
  https://data.lacity.org/A-Well-Run-City/Parking-Citations/wjz9-h9np
  
This code needs the raw data to be named 'agency_codes.tsv'
and to be in the same folder as the jupyter notebook
'''

agency_df_raw = pd.read_csv('agency_codes.tsv',sep='\t',header=0)

agency_df=agency_df_raw

agency_df.columns=['Agency','agency_name','agency_shortname']
agency_df.head()

,Agency,agency_name,agency_shortname
0,1,WESTERN,WESTERN
1,2,LAX CURRENT,LAX CUR
2,3,VALLEY,VALLEY
3,4,HOLLYWOOD,HOLLYWOOD
4,5,SOUTHERN,SOUTHERN


In [9]:
########################################
# Cleaning Data
########################################
df = df_raw

########################################
# Adding Data
########################################
df['one']=1 #used instead of ticket number to count rows
df = pd.merge(df,agency_df,how='left',on=['Agency']) #adding agency name

########################################
# Removing Unnecessary Data
########################################
df = df[df['Issue Date'].notnull()] #removing anything with an unknown issue date
df = df[df['Fine amount'].notnull()] #removing anything with an unknown fine amount
del df['VIN'] #If these are actually VINs, I'm not sure if they should be released publicly in the first place
del df['Meter Id'] #no need for this
del df['Marked Time'] #not sure what this is. Don't need it
del df['Ticket number'] # no need to keep this right now since it's just a unique value per now
del df['Route'] #don't know how I'd use this
del df['Violation code'] #I can use the violation name instead
del df['Plate Expiry Date'] #I'm going to leave this out to keep things simple

#Lat/Lon is in US Feet coordinates according to the NAD1983StatePlaneCaliforniaVFIPS0405_Feet projection
#Removing for now
del df['Latitude']
del df['Longitude']

del df['Location'] #I would need a serious geocoding package to make use of this partial address --might be able to use the Google Geocoding API
del df['Agency'] #No need for Agency Code now
del df['agency_shortname'] #No need for agency shortname right now

########################################
# Replacing Nulls
########################################

#this part is going to make it easier when I do the group bys later so we can include the nulls
df['Issue time'].fillna(value = -999,inplace = True,axis=0) #replacing nulls with -999
df['RP State Plate'].fillna(value = 'Unknown',inplace = True,axis=0) #replacing nulls with 'Unknown'
df['Make'].fillna(value = 'Unknown',inplace = True,axis=0) #replacing nulls with 'Unknown'
df['Body Style'].fillna(value = 'Unknown',inplace = True,axis=0) #replacing nulls with 'Unknown'
df['Color'].fillna(value = 'Unknown',inplace = True,axis=0) #replacing nulls with 'Unknown'
df['Violation Description'].fillna(value = 'Unknown',inplace = True,axis=0) #replacing nulls with 'Unknown'
df['agency_name'].fillna(value = 'Unknown',inplace = True,axis=0) #replacing nulls with 'Unknown'

########################################
# Time
########################################

# Time
df['Issue time']=df['Issue time'].astype(int)
df['Issue time']=df['Issue time'].apply('{0:0>4}'.format) #nulls will come out as '-999'
df['Issue time']=df['Issue time'].astype(str)

df['issue_hour']=df['Issue time'].str[:2] #nulls will show up as '-9'

'''The next two lines of commented code are for 
minute-level information.  I'm excluding it for now'''
#df['issue_minute']=df['Issue time'].str[2:4]
#df['issue_h_m']=df['issue_hour']+':'+df['issue_minute']+':00'

########################################
# Last Deletes for Space
########################################
'''
These fields were deleted to save space.
Tableau Public can only handle 1M rows
'''

del df['Issue time'] #too much granularity
del df['RP State Plate'] #mostly california
del df['Make'] #I can do without this
del df['Body Style']
del df['Color'] 
del df['agency_name']
########################################
# Renaming Columns
########################################
df.rename(
        columns ={
            'Issue Date':'issue_date'
            ,'Issue time':'issue_time'
            ,'RP State Plate':'state_plate'
            ,'Body Style':'veh_body'
            ,'Violation Description':'violation'
            ,'Fine amount':'fine'
                }
        ,inplace = True)

print 'Total Rows: %d' % (df.one.sum())
print
print df.count()
print
print
print df.info()
print
df.head(20)

Total Rows: 7791341

issue_date    7791341
violation     7791341
fine          7791341
one           7791341
issue_hour    7791341
dtype: int64


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7791341 entries, 0 to 7797439
Data columns (total 5 columns):
issue_date    object
violation     object
fine          float64
one           int64
issue_hour    object
dtypes: float64(1), int64(1), object(3)
memory usage: 356.7+ MB
None



,issue_date,violation,fine,one,issue_hour
0,12/30/2015,RED ZONE,93.0,1,22
1,12/30/2015,WHITE ZONE,58.0,1,22
2,12/30/2015,DISPLAY OF TABS,25.0,1,17
3,12/30/2015,METER EXP.,63.0,1,17
4,12/30/2015,PREFERENTIAL PARKING,68.0,1,18
5,12/30/2015,NO PARK/STREET CLEAN,73.0,1,05
6,01/04/2016,NO PARKING,73.0,1,12
11,12/30/2015,PREFERENTIAL PARKING,68.0,1,18
14,01/05/2016,RED ZONE,93.0,1,14
15,12/29/2015,22500H,68.0,1,10


In [10]:
#######################################################
# Aggregating Cleaned Data - reducing size of dataframe
#######################################################
col = df.columns.tolist()
col.remove('one')
df_agg=df.groupby(col).one.sum().reset_index()
print df_agg.nunique()
print df_agg.count()
df_agg.head()

issue_date    1623
violation      339
fine            38
issue_hour      25
one            923
dtype: int64
issue_date    693478
violation     693478
fine          693478
issue_hour    693478
one           693478
dtype: int64


,issue_date,violation,fine,issue_hour,one
0,01/01/2014,NO EVIDENCE OF REG,50.0,13,1
1,01/01/2014,NO EVIDENCE OF REG,50.0,15,1
2,01/01/2015,18 IN. CURB/1 WAY,63.0,10,1
3,01/01/2015,18 IN. CURB/2 WAY,63.0,02,1
4,01/01/2015,18 IN. CURB/2 WAY,63.0,07,1


In [ ]:
########################################
# Writing to tab separated CSV
########################################
'''This will write the csv file to the same
folder as the jupyter notebook is in'''

df_agg.to_csv('tableauData.csv',sep='\t')